In [ ]:
from pathlib import Path

import flopy
import pastas as ps
from pastas_plugins import modflow as ppmf
import pandas as pd

bindir = Path("bin")
if not (bindir / "mf6").exists():
    bindir.mkdir(parents=True, exist_ok=True)
    flopy.utils.get_modflow(bindir, repo="modflow6")

In [ ]:
data = ps.load_dataset("collenteur_2021")

head = wagna = data["wagna"]["head [m]"].dropna().rename("head")
evap = data["wagna"]["evaporation [mm/d]"].multiply(1e-3).rename("evaporation").dropna()
prec = (
    data["wagna"]["precipitation [mm/d]"]
    .multiply(1e-3)
    .rename("precipitation")
    .loc[evap.index]
)

In [ ]:
ml = ps.Model(head, name="mftest")
mfml = ppmf.ModflowModel(
    ml,
    exe_name="bin/mf6",
    sim_ws="mftest",
    silent=True,
)
rch = ppmf.ModflowRch(prec, evap)
ghb = ppmf.ModflowGhb()
mfml.add_modflow_package([rch, ghb])
ml.add_stressmodel(mfml)
mfml.simulate(p=ml.parameters.initial.values)
# ml.simulate(p=ml.parameters.initial.values) # this does not work?
# TypeError: ModflowModel.simulate() takes 2 positional arguments but 6 were given